# Profiling Code Execution with cProfile


In [1]:
import cProfile, pstats, io
from pstats import SortKey
import pandas as pd
import time
from signal_processing_utilities import process_signal

from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader

# Custom import of local file "decode"
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

# Custom import of local file "encode"
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

In [2]:
def print_full(data_frame):
    """This function will display all rows of the dataframe.

    Args:
        data_frame (DataFrame): This is a pandas dataframe of
                                arbitrary length.
    """
    pd.set_option("display.max_rows", len(data_frame))
    pd.set_option("display.width", 1000)
    print(data_frame)
    pd.reset_option("display.max_rows")
    pd.reset_option("display.width")

In [3]:
file = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav"
compressed_file_path = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav.brainwire"
decompressed_file_path = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav.copy"

In [11]:
parser = encode.initialize_argument_parser()
args = parser.parse_args([file, compressed_file_path, "-m=u"])
start_time = time.time_ns()
encode.main(args)
stop_time = time.time_ns()
process_signal.print_time_each_function_takes_to_complete_processing(
    start_time, stop_time, executed_line="unique huffman compression"
)
process_signal.print_size_of_file_compression(
    file_path=file, compressed_file_path=compressed_file_path
)


decode_parser = decode.initialize_argument_parser()
decode_args = decode_parser.parse_args([compressed_file_path, decompressed_file_path])
start_time = time.time_ns()
decode.main(decode_args)
stop_time = time.time_ns()
process_signal.print_time_each_function_takes_to_complete_processing(
    start_time, stop_time, executed_line="decode unique huffman compression"
)
process_signal.print_size_of_file_compression(
    file_path=file, compressed_file_path=compressed_file_path
)


Executed Line: unique huffman compression...
Time Δ Nanoseconds: 149207000
Time Δ Microseconds: 149207.0
Time Δ Milliseconds: 149.207
Time Δ Seconds: 0.149207


Original File Size: 197526
Compressed File Size: 74054
Percent of Compression: 62.51%
Percent of Compressed File Size Relative to Required File Size: 7502.938%

Executed Line: decode unique huffman compression...
Time Δ Nanoseconds: 1968372000
Time Δ Microseconds: 1968372.0
Time Δ Milliseconds: 1968.372
Time Δ Seconds: 1.968372


Original File Size: 197526
Compressed File Size: 74054
Percent of Compression: 62.51%
Percent of Compressed File Size Relative to Required File Size: 7502.938%


In [ ]:
pr = cProfile.Profile()
pr.enable()
encode.main(args)
s = io.StringIO()
sortby = SortKey.TIME
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

In [ ]:
s.getvalue()

In [7]:
with cProfile.Profile() as pr:

    encode.main(args)

df = pd.DataFrame(
    pr.getstats(),
    columns=["func", "ncalls", "ccalls", "tottime", "cumtime", "callers"],
)

In [ ]:
df[df["cumtime"] == df["cumtime"].max()]

In [ ]:
df.sort_values("tottime", ascending=False)

In [ ]:
df.iloc(1)

In [ ]:
print_full(df.sort_values("tottime", ascending=False))